# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
from sklearn import linear_model, metrics

In [3]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [4]:
# frequency table for prestige and whether or not someone was admitted

#print df['prestige'].value_counts()
#print df['admit'].value_counts()

pd.crosstab(df['prestige'], df['admit'])

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [5]:
prestige_dummy = pd.get_dummies(df.prestige)

prestige_dummy.columns = ['prestige_' + str(i) for i in prestige_dummy.columns]

#join the new dummy variables back into the larger dataset
prestige_model_data = df.join(prestige_dummy)

prestige_model_data.head()

,admit,gre,gpa,prestige,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,3.0,0.0,0.0,1.0,0.0
1,1,660.0,3.67,3.0,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,1.0,0.0,0.0,0.0
3,1,640.0,3.19,4.0,0.0,0.0,0.0,1.0
4,0,520.0,2.93,4.0,0.0,0.0,0.0,1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: r-1, where r is the number of categories in the categorical variable (3 in this case)

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [6]:
cols_to_keep = ['admit', 'gre', 'gpa', 'prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']
handCalc = prestige_model_data[cols_to_keep]
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


In [7]:
# crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

pd.crosstab(handCalc['admit'], handCalc['prestige_1.0'])

prestige_1.0,0.0,1.0
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [8]:
#Odds = number of successes / number of failures 
#Odds exposed

odds_exposed = 33/28.0

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [9]:
odds_unexposed = 93/243.0

#### 3.3 Calculate the odds ratio

In [10]:
odds_ratio = odds_exposed/odds_unexposed

print odds_ratio

3.07949308756


#### 3.5 Print the cross tab for prestige_4

In [11]:
pd.crosstab(handCalc['admit'], handCalc['prestige_4.0'])

prestige_4.0,0.0,1.0
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [12]:
#the odds of being admitted to grad school if you attended a #4 ranked college
odds_exposed = 12/55.0

#the odds of admission if you did not attend a #4 ranked college
odds_unexposed = 114/216.0

odds_ratio = odds_exposed/odds_unexposed

print odds_ratio

0.413397129187


#### 3.7 Write this finding in a sentence

Answer: The odds of being admitted to graduate school are 60% lower for students who attended a #4 ranked school compared to students who did not attend a #4 ranked school. 

OR

For students who attended a #4 ranked school the odds of being admitted into gradauate school are 0.41 times the odds of students who did not attend a #4 ranked school.

## Part 4. Analysis

In [13]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
new_data = handCalc[cols_to_keep].join(handCalc.ix[:, 'prestige_2.0':])

new_data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0.0,1.0,0.0
1,1,660.0,3.67,0.0,1.0,0.0
2,1,800.0,4.00,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [14]:
# manually add the intercept
new_data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [15]:
train_cols = new_data.columns[1:]

logit = sm.Logit(new_data['admit'], new_data[train_cols])

#### 4.2 Fit the model

In [16]:
result = logit.fit()

#Closer you get to 1 the better (current function value)

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [17]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 01 Aug 2016   Pseudo R-squ.:                 0.08166
Time:                        20:40:16   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [21]:
#Calculating odds ratios of the coefficients

#print result.params

print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [20]:
# odds ratios and 95% CI

params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: If you attended a #2 ranked school, your chances of being admitted will decrease by 50%.

#### 4.6 Interpret the OR of GPA

Answer: For every 2.8 points your GPA is higher, your chances of being admitted increase by 1%.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [34]:
#Creating all possible combos

def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [36]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(new_data['gre'].min(), new_data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(new_data['gpa'].min(), new_data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [44]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']

dummie_values = pd.get_dummies(combos['prestige'], prefix='prestige')

dummie_values.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

# keep only what we need for making predictions

cols_to_keep = combos.columns

combos = combos[cols_to_keep].join(dummie_values.ix[:,'prestige_2.0':])

#### 5.2 Make predictions on the enumerated dataset

In [52]:
combos['admit_predictions'] = result.predict(combos[train_cols])

print combos.tail()

       gre       gpa  prestige  intercept  prestige_2.0  prestige_3.0  \
395  800.0  3.806667       4.0        1.0           0.0           0.0   
396  800.0  4.000000       1.0        1.0           0.0           0.0   
397  800.0  4.000000       2.0        1.0           1.0           0.0   
398  800.0  4.000000       3.0        1.0           0.0           1.0   
399  800.0  4.000000       4.0        1.0           0.0           0.0   

     prestige_4.0  admit_predictions  
395           1.0           0.334286  
396           0.0           0.734040  
397           0.0           0.582995  
398           0.0           0.419833  
399           1.0           0.368608  


#### 5.3 Interpret findings for the last 4 observations

Answer: 

1) If you had a 800 GRE score, 4.0 GPA and attended a #1 ranked prestige school, your chances of getting admitted are 73%.

2) If you had a 800 GRE score, 4.0 GPA and attended a #2 ranked prestige school, your chances of getting admitted are 58%.

3) If you had a 800 GRE score, 4.0 GPA and attended a #3 ranked prestige school, your chances of getting admitted are 42%.

4) If you had a 800 GRE score, 4.0 GPA and attended a #4 ranked prestige school, your chances of getting admitted are 37%.


## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.